In [6]:

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score,train_test_split
import torch
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.mlls import ExactMarginalLogLikelihood

from gpytorch.distributions import MultivariateNormal
from gpytorch.likelihoods import GaussianLikelihood

import torch
from linear_operator import to_dense
from gpytorch.constraints import Positive
from gpytorch.kernels import Kernel
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
from sklearn.preprocessing import StandardScaler


from sklearn.datasets import load_diabetes
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Load data

# diabetes = load_diabetes()
# X, y = diabetes.data, diabetes.target
data = fetch_california_housing()
X, y = data.data, data.target
X = X[:210, :]
y = y[:210]
# X = X[:, :3]

# Split the data into training and testing sets
train_x, test_x, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 20% data as test
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
train_x = scaler.fit_transform(train_x)

# Transform the test data using the same scaler
test_x = scaler.transform(test_x)

# Convert to torch tensors
train_x = torch.tensor(train_x,dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)






'''
k(x_1, x_2) = \exp\left(-\frac{1}{2 \l^2} \sum_{k=1}^d \left(\left| x_{1k} - x_{2k} \right|\right)^2 \right)
'''
    
class CustomRBFKernel(gpytorch.kernels.Kernel):

    has_lengthscale = True

    def forward(self, x1, x2, diag=False, **params):
        # Compute squared distance
        # squared_dist = self.covar_dist(x1, x2, square_dist=True, diag=diag, **params)
        diff = x1.unsqueeze(1) - x2.unsqueeze(0)
        diff = torch.abs(diff)
        squared_dist = (diff ** 2).sum(-1)

        # Divide by 2 * lengthscale^2
        scaled_squared_dist = squared_dist.div(2 * self.lengthscale.pow(2)) #.div

        # Compute exponential
        covar_matrix = scaled_squared_dist.mul_(-1).exp_()

        return covar_matrix


    

class DPkernel(gpytorch.kernels.Kernel):
    def __init__(self, base_kernel, num_dims, q_additivity, train_x, **kwargs):
        super().__init__(**kwargs)
        self.base_kernel = base_kernel
        self.num_dims = num_dims
        self.q_additivity = q_additivity
        self.register_parameter(
            name="raw_outputscale", 
            parameter=torch.nn.Parameter(torch.zeros(1, self.q_additivity))
        )
        self.outputscale_constraint = gpytorch.constraints.Positive()
        self.register_constraint("raw_outputscale", self.outputscale_constraint)
        
        # Extract mean and standard deviation
        self.mu = torch.mean(train_x, dim=0)  # Mean of train_x
        self.delta = torch.std(train_x, dim=0)  # Standard deviation of train_x

    @property
    def outputscale(self):
        return self.outputscale_constraint.transform(self.raw_outputscale).squeeze()

    @outputscale.setter
    def outputscale(self, value):
        if not torch.is_tensor(value):
            value = torch.tensor(value, device=self.raw_outputscale.device)
        self.initialize(raw_outputscale=self.outputscale_constraint.inverse_transform(value))

    def forward(self, x1, x2, diag=False, **params):
        x1_size = x1.size(0)
        x2_size = x2.size(0)
        
        result = torch.zeros(x1_size, x2_size, device=x1.device)
        sum_order_b = torch.zeros(x1_size, x2_size, device=x1.device)
        kernels = []

        # First order calculations with additional term
        for d in range(self.num_dims):
            x1_d = x1[:, d:d+1]
            x2_d = x2[:, d:d+1]
            k_d = self.base_kernel(x1_d, x2_d).evaluate()
            kernels.append(k_d)
            sum_order_b += k_d

            # Additional term based on the formula form the paper Additice Gaussian Process
            diff = x1_d.unsqueeze(1) - x2_d.unsqueeze(0)
            squared_dist = (diff ** 2).sum(-1)
            scaled_squared_dist = squared_dist.div(2 * self.base_kernel.lengthscale.pow(2))

            mu_d = self.mu[d]
            delta_d = self.delta[d]
            mu_term = ((x1_d - mu_d) ** 2).sum(-1, keepdim=True) + ((x2_d - mu_d) ** 2).sum(-1)
            mu_term = mu_term.div(2 * (self.base_kernel.lengthscale.pow(2) + delta_d.pow(2)))

            exp_term_1 = (-scaled_squared_dist).exp()
            exp_term_2 = (-mu_term).exp()

            additional_term = self.base_kernel.lengthscale.sqrt() * (self.base_kernel.lengthscale.pow(2) + 2 * delta_d.pow(2)).sqrt()
            additional_term = additional_term.div(self.base_kernel.lengthscale.pow(2) + delta_d.pow(2))
            additional_term = additional_term * exp_term_2

            sum_order_b += exp_term_1 - additional_term
        
        outputscale = self.outputscale.unsqueeze(0) if len(self.outputscale.shape) == 0 else self.outputscale
        result += sum_order_b * self.outputscale[0]

        # Compute higher order interactions
        for i in range(1, self.q_additivity):
            temp_sum = torch.zeros(x1_size, x2_size, device=x1.device)
            new_kernels = []
            for j in range(self.num_dims):
                for k in range(j + 1, self.num_dims):
                    new_kernel = kernels[j] * kernels[k]
                    new_kernels.append(new_kernel)
                    temp_sum += new_kernel

            kernels = new_kernels
            result += temp_sum * self.outputscale[i]

        return result
    
class DPkernel(gpytorch.kernels.Kernel):
    def __init__(self, base_kernel, num_dims, q_additivity, train_sample, **kwargs):
        super().__init__(**kwargs)
        self.base_kernel = base_kernel
        self.num_dims = num_dims
        self.q_additivity = q_additivity
        self.train_sample = train_sample
        self.register_parameter(
            name="raw_outputscale", 
            parameter=torch.nn.Parameter(torch.zeros(1, self.q_additivity))
        )
        self.outputscale_constraint = gpytorch.constraints.Positive()
        self.register_constraint("raw_outputscale", self.outputscale_constraint)
        
        # Extract mean and standard deviation
        self.mu = torch.mean(self.train_sample, dim=0)  # Mean of train_x
        self.delta = torch.std(self.train_sample, dim=0)  # Standard deviation of train_x

    @property
    def outputscale(self):
        return self.outputscale_constraint.transform(self.raw_outputscale).squeeze()

    @outputscale.setter
    def outputscale(self, value):
        if not torch.is_tensor(value):
            value = torch.tensor(value, device=self.raw_outputscale.device)
        self.initialize(raw_outputscale=self.outputscale_constraint.inverse_transform(value))

    def forward(self, x1, x2, diag=False, **params):
        x1_size = x1.size(0)
        x2_size = x2.size(0)
        
        result = torch.zeros(x1_size, x2_size, device=x1.device)
        sum_order_b = torch.zeros(x1_size, x2_size, device=x1.device)
        kernels = []

        # First order calculations with additional term
        for d in range(self.num_dims):
            x1_d = x1[:, d:d+1]
            x2_d = x2[:, d:d+1]
            # first term form base kernel
            k_d = self.base_kernel(x1_d, x2_d).evaluate()
            # kernels.append(k_d)
            # sum_order_b += k_d

            # Additional term based on the formula in the image for the first order
            #first term
            

            #second term
            mu_d = self.mu[d]
            delta_d = self.delta[d]
            mu_term = ((x1_d - mu_d) ** 2).sum(-1, keepdim=True) + ((x2_d - mu_d) ** 2).sum(-1)
            mu_term = mu_term.div(2 * (self.base_kernel.lengthscale.pow(2) + delta_d.pow(2)))
            exp_term_2 = (-mu_term).exp()

            middle_term = self.base_kernel.lengthscale * torch.sqrt(self.base_kernel.lengthscale.pow(2) + 2 * delta_d.pow(2)).sqrt()
            middle_term = middle_term.div(self.base_kernel.lengthscale.pow(2) + delta_d.pow(2))
            additional_term = middle_term * exp_term_2

            k0 = k_d - additional_term
            kernels.append(k0)

            sum_order_b += k0
        
        outputscale = self.outputscale.unsqueeze(0) if len(self.outputscale.shape) == 0 else self.outputscale
        result += sum_order_b * self.outputscale[0]

        # Compute higher order interactions
        for i in range(1, self.q_additivity):
            temp_sum = torch.zeros(x1_size, x2_size, device=x1.device)
            new_kernels = []
            for j in range(self.num_dims):
                for k in range(j + 1, self.num_dims):
                    new_kernel = kernels[j] * kernels[k]
                    new_kernels.append(new_kernel)
                    temp_sum += new_kernel

            kernels = new_kernels
            result += temp_sum * self.outputscale[i]

        return result


# class DPkernel(gpytorch.kernels.Kernel):
#     def __init__(self, base_kernel, num_dims, q_additivity, **kwargs):
#         super().__init__(**kwargs)
#         self.base_kernel = base_kernel
#         self.num_dims = num_dims
#         self.q_additivity = q_additivity
#         self.register_parameter(
#             name="raw_outputscale", 
#             parameter=torch.nn.Parameter(torch.zeros(1, self.q_additivity))
#         )
#         self.outputscale_constraint = gpytorch.constraints.Positive()
#         self.register_constraint("raw_outputscale", self.outputscale_constraint)

#     @property
#     def outputscale(self):
#         return self.outputscale_constraint.transform(self.raw_outputscale).squeeze()

#     @outputscale.setter
#     def outputscale(self, value):
#         if not torch.is_tensor(value):
#             value = torch.tensor(value, device=self.raw_outputscale.device)
#         self.initialize(raw_outputscale=self.outputscale_constraint.inverse_transform(value))

#     def forward(self, x1, x2, diag=False, **params):
#     # Determine sizes based on input matrices
#         x1_size = x1.size(0)
#         x2_size = x2.size(0)
        
#         # Initialize matrices based on input sizes
#         result = torch.zeros(x1_size, x2_size, device=x1.device) #initialize the result matrix
#         sum_order_b = torch.zeros(x1_size, x2_size, device=x1.device) # initialize the matrix for the matrix for a single order
#         kernels =[] # list were the z1, z2,... would be stored

#         # print(f"Initial x1 shape: {x1.shape}, x2 shape: {x2.shape}")
        
#         #calculations for first order
#         #calcualte the kernels for each dimentions
#         for d in range(self.num_dims):
#             x1_d = x1[:, d:d+1]
#             x2_d = x2[:, d:d+1]
#             k_d = self.base_kernel(x1_d, x2_d).evaluate() # change thek to k0
            
#             kernels.append(k_d) #save them in order in the kernels list
#             # print(f"Kernel k_d at dim {d} shape: {k_d.shape}, sum_order_b shape: {sum_order_b.shape}")

#             sum_order_b += k_d # add each one dimension kernels to one matrix for first order
    
#         # first_kernels = kernels
#         outputscale = self.outputscale.unsqueeze(0) if len(self.outputscale.shape) == 0 else self.outputscale
#         result += sum_order_b * self.outputscale[0] #add the first order kernel miltiplied by first outputscale

#         # Compute higher order interactions
#         for i in range(1, self.q_additivity):
#             temp_sum = torch.zeros(x1_size, x2_size, device=x1.device)
#             new_kernels = []
#             for j in range(self.num_dims):
#                 for k in range(j + 1, self.num_dims):
#                     new_kernel = kernels[j] * kernels[k]
#                     new_kernels.append(new_kernel)
#                     temp_sum += new_kernel

#             kernels = new_kernels  # update kernels list with new order interactions
#             result += temp_sum * self.outputscale[i]

#         return result

# Example usage in a GP model
class MyGP(gpytorch.models.ExactGP): # i need to find a diferent model
    def __init__(self, train_x, train_y, likelihood):
        super(MyGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        # self.base_kernel = gpytorch.kernels.RBFKernel()
        self.base_kernel = CustomRBFKernel()
        self.covar_module = DPkernel(base_kernel=self.base_kernel, num_dims=train_x.size(-1), q_additivity=train_x.size(-1), train_sample = train_x)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)  # Make sure to pass x twice WHY
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Create the GP model
likelihood = gpytorch.likelihoods.GaussianLikelihood()


model = MyGP(train_x, y_train.squeeze(-1), likelihood)
model.eval()
# with torch.no_grad():
#     untrained_pred_dist = likelihood(model(test_x))
#     predictive_mean = untrained_pred_dist.mean
#     lower, upper = untrained_pred_dist.confidence_region()
# Set up optimizer and marginal log likelihood
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

model.train()
likelihood.train()
# Training loop
training_iter = 1000
for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    # print(output)
    loss = -mll(output, y_train)
    loss = loss.mean() 
    loss.backward()
    # print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
    #     i + 1, training_iter, loss.item(),
    #     model.covar_module.base_kernel.lengthscale.item(),
    #     model.likelihood.noise.item()
    # ))
    optimizer.step()
# print('likelihood noise', likelihood.noise)
# print('likelihood noise raw', likelihood.noise_covar.raw_noise)
model.eval()
# with torch.no_grad():
#     trained_pred_dist = likelihood(model(test_x))
#     predictive_mean = trained_pred_dist.mean
#     lower, upper = trained_pred_dist.confidence_region()
# Viewing model parameters after training
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.data}')


#evaluating there is a problem when the test_y and test_x have float numbers
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode (mode is for computing predictions through the model posterior.)
    likelihood.eval()
    output = likelihood(model(test_x))  # Make predictions on new data 
    


for constraint_name, constraint in model.named_constraints():
    print(f'Constraint name: {constraint_name:55} constraint = {constraint}')


# Extracting means and standard deviations
predicted_means = output.mean.numpy() 
predicted_stddevs = output.stddev.numpy()  # Extract standard deviations

print("Predicted Means:")
print(predicted_means)

print("Predicted Standard Deviations:")
print(predicted_stddevs)


print("x_train", train_x.shape)

Parameter name: likelihood.noise_covar.raw_noise           value = tensor([-3.1533])
Parameter name: base_kernel.raw_lengthscale                value = tensor([[0.6244]])
Parameter name: covar_module.raw_outputscale               value = tensor([[-0.7119, -4.8627, -2.2748, -8.3249, -9.2394, -9.0218, -7.6204, -2.4694]])
Constraint name: likelihood.noise_covar.raw_noise_constraint             constraint = GreaterThan(1.000E-04)
Constraint name: base_kernel.raw_lengthscale_constraint                  constraint = Positive()
Constraint name: covar_module.raw_outputscale_constraint                 constraint = Positive()
Predicted Means:
[1.0501919  1.5123014  1.0332913  0.7673197  1.0754166  2.9719605
 1.9691772  1.5605273  2.5621562  1.9208918  3.3095746  1.3127961
 1.3040628  3.264596   1.5741014  0.41906548 2.1697662  1.6299438
 1.458086   1.2952032  1.6194601  0.8747935  1.8426151  2.360249
 2.461608   3.5174327  1.1199017  3.1491973  1.9761467  1.5538125
 2.8810065  0.63751507 2.62011

In [7]:


# calculate teh alpha_hat_eta
model.eval()
likelihood.eval()
with torch.no_grad():
    # Evaluate the kernel matrix
    t_k_matrix = model.covar_module(train_x).evaluate()
    
    # Ensure the noise variance is non-zero and sufficiently large to avoid singularity
    noise_variance = likelihood.noise_covar.noise if likelihood.noise_covar.noise > 1e-6 else 1e-6
    n_matrix = torch.eye(t_k_matrix.size(-1), device=t_k_matrix.device) * noise_variance
    
    # Add regularization to avoid singular matrix

    K_inv = torch.inverse(t_k_matrix + n_matrix)# + torch.eye(t_k_matrix.size(-1), device=t_k_matrix.device))

    # Compute alpha_hat_eta using the inverse (dot product)
    alpha_hat_eta = torch.matmul(K_inv, y_train).unsqueeze(1)

    print(alpha_hat_eta.shape)

torch.Size([168, 1])


In [11]:
n, d = train_x.shape
model.eval()
likelihood.eval()

with torch.no_grad():
    kernel = model.covar_module

    # Initialize the matrix K with zeros
    K_per_feature = torch.zeros((n, d))

    # Extracting a specific instance's features
    instance_features = train_x[3].unsqueeze(0)  # Shape (1, d)

    # Loop over each feature dimension
    for i in range(d):
        # Reshape the specific feature across all samples to match the input shape required by the kernel
        feature_column = train_x[:, i].unsqueeze(1)  # Shape (n, 1)
        instance_feature = instance_features[:, i].unsqueeze(1)  # Shape (1, 1)

        K_per_feature[:, i] = kernel(instance_feature, feature_column).evaluate().squeeze()
        # print(K_per_feature[:, i])

print("Matrix K (n*d):")
print(K_per_feature.shape)

Matrix K (n*d):
torch.Size([168, 8])


# Brute Force

In [12]:
from itertools import chain, combinations


n_samples, n_features = train_x.size()
val = torch.zeros(n_features)

# Calculate the extended K
temp = range(n_features)
feature_combinations = list(chain.from_iterable(combinations(temp, r) for r in range(n_features+1)))
feature_combinations.remove(())

extended_K = torch.zeros((n_samples, len(feature_combinations)))
for i, comb in enumerate(feature_combinations):
    product = torch.ones(n_samples)#.unsqueeze(0)

    for j in comb:
        product = product *K_per_feature[:, j]#.unsqueeze(0)
    extended_K[:, i] = product

# Create a loop for each feature to compute its Shapley value
for j in range(n_features):
    # Find the subsets where the j-th feature was used
    indices_of_kj_columns = [idx for idx, combination in enumerate(feature_combinations) if j in combination]
    indices_of_kj_columns = torch.tensor(indices_of_kj_columns) # Convert the list to a tensor

    # Update the extended_K matrix to only include the columns with the indices found
    updated_extended_K = torch.zeros(n_samples, len(indices_of_kj_columns))
    for i, idx in enumerate(indices_of_kj_columns):
        updated_extended_K[:, i] = extended_K[:, idx]

    # Create a vector of weights for the corresponding columns
    weights = torch.zeros((len(indices_of_kj_columns), 1))

    # Set the weights as 1 / length of the combination
    for i, idx in enumerate(indices_of_kj_columns):
        weights[i] = 1 / len(feature_combinations[idx])
        

    # Compute omega
    omega = torch.matmul(updated_extended_K, weights)
    # omega = torch.zeros(n_samples,1)
    # for i in range(len(indices_of_kj_columns)):
    #     col = (updated_extended_K[:, i] * weights[i]).unsqueeze(1)
        
    
    #     omega += col

    # Compute the Shapley value for the j-th feature
    val[j] = torch.matmul(omega.T, alpha_hat_eta)

print('This is the Shapley value:', val)


This is the Shapley value: tensor([148.7976, 144.6180, 146.8477, 135.9618, 155.0772, 141.9465, 153.5995,
        132.3171])


# DP

In [13]:
n_samples, n_features = train_x.size()
def Omega(X, i, q_additivity=None, feature_type='numerical'):
    
    
    if q_additivity is None:
        q_additivity = d
    
    # Reorder columns so that the i-th column is first
    idx = torch.arange(d)
    idx[i] = 0
    idx[0] = i
    X = X[:, idx]

    # Initialize dp array
    dp = torch.zeros((q_additivity, d, n))

    # Initial sum of features across the dataset
    sum_current = torch.zeros((n,))
    
    # Fill the first order dp (base case)
    for j in range(d):
        dp[0, j, :] = X[:, j]
        sum_current += X[:, j]

    # Fill the dp table for higher orders
    for i in range(1, q_additivity):
        temp_sum = torch.zeros((n,))
        for j in range(d):
            # Subtract the previous contribution of this feature when moving to the next order
            sum_current -= dp[i - 1, j, :]
            dp[i, j, :] = (i / (i + 1)) * (X[:,j]* sum_current)
            temp_sum += dp[i, j, :]
        
        sum_current = temp_sum

    # Sum up all contributions from the first dimension of each feature to get the final values
    omega = torch.sum(dp[:, 0, :], axis=0)

    return omega, dp


val = torch.zeros(n_features)
for i in range(n_features):
    omega_dp, _ = Omega(K_per_feature, i, q_additivity=None, feature_type='numerical')
    val[i] = torch.matmul(omega_dp, alpha_hat_eta)

print('This is val:', val)

This is val: tensor([148.7844, 144.6147, 146.8480, 135.9597, 155.0752, 141.9478, 153.5898,
        132.3176])
